In [3]:
from pygeodesy import sphericalTrigonometry as st
import geojson
import folium
from folium.features import DivIcon

In [5]:
polygon_file = "data/open-polygon.geojson"
points_file = "data/points.geojson"

with open(polygon_file) as f:
    data_poly = geojson.loads(f.read())
with open(points_file) as f:
    data_pnts = geojson.loads(f.read())


FileNotFoundError: [Errno 2] No such file or directory: 'data/open-polygon.geojson'

In [ ]:
def transform_data(data):
    # swap order of coords from [Long,Lat] to [Lat,Long]
    pnts = [[p[1],p[0]] for p in
           data.features[0].geometry.coordinates]
    # prepare data type appropriate for 'nearestOn3' method
    pnts_LL = [st.LatLon(p[0],p[1]) for p in pnts]
    return pnts, pnts_LL
 ​
pnts, pnts_LL = transform_data(data_pnts)
poly, poly_LL = transform_data(data_poly)

In [ ]:
nearest_pnts = [st.nearestOn3(pnt_LL, poly_LL) for pnt_LL in pnts_LL]


In [ ]:
n_pnts = [[np[0].lat,np[0].lon] for np in nearest_pnts]

In [ ]:
poi = data_pnts.features[0].geometry.coordinates[4]
poi = [poi[1], poi[0]] # in folium.Map the order is - Lat, Long
 ​
## Display polygon on the map
m = folium.Map(location=poi, zoom_start=9, tiles='openstreetmap')
folium.GeoJson(data_poly).add_to(m) # here the order is - Long, Lat
 ​
## Display reference points 'pnts' and nearest points 'n_pnts'
def show_pnts_on_map(points,map,color='red'):
    fstr = '<div style="font-size: 16pt; color: ' + color + '">{} </div>'
    for i,_ in enumerate(points):
        folium.CircleMarker(location=points[i], color=color,
            radius=5, fill=color).add_to(map)
        folium.map.Marker(location=points[i],
            icon=DivIcon(icon_size=(150, 36),icon_anchor=(0, 0),
            html=fstr.format(str(i+1)))).add_to(map)
     return map
 ​
 m = show_pnts_on_map(pnts,m,'red')
 m = show_pnts_on_map(n_pnts,m,'blue')
 ​
 ## Display line segments connecting reference points and corresponding nearest points
 for i,_ in enumerate(pnts):
     folium.PolyLine(locations=[pnts[i],n_pnts[i]], color='red').add_to(m)
     print(pnts[i][0], pnts[i][1], n_pnts[i][0], n_pnts[i][1],
           nearest_pnts[i][1],nearest_pnts[i][2])
 ​
 ## Store the map on the disk
 m.save('map_my_1.html')